In [3]:
import numpy as np
from collections import defaultdict

In [4]:
class SparseMatrix():
    """
    Class SparseMatrix
    
    Methods:
    sparse_time_sparse(A,B) -> SparseMatrix
    pow_sparse(self,k) -> SparseMatrix
    sparse_times_dense(self,other) -> numpy.array
    cholesky_decomp(A) -> SparseMatrix
    sparse_transpose(self) -> SparseMatrix
    slice_sparse(self,rows,columns) -> SparseMatrix
    solve_sparse(A,B) -> numpy.array
    cheaters_LU(self) -> tuple["SparseMatrix",np.array]
    """
    def __init__(self, values: np.array, columns: np.array, rows:np.array, realsize: int):
        """
        v: non-zero values of the matrix
        c: columns for each element of v
        r: rows for each element of v
        rs: n for a assumed nxn matrix
        """
        self.v = values
        self.c = columns
        self.r = rows
        self.n = len(values)
        self.rs = realsize


    def sparse_times_sparse(A: "SparseMatrix", B: "SparseMatrix") -> "SparseMatrix":
        """
        Inputs:
        A,B: Sparse matrices
        ___________________________
        Output:
        Sparse matrix result of the multiplication

        Note: this assumes that the matrices can be multiplied
        """

        #create a dictionary that maps all the values and columns of other to their rows
        other_dict: dict = {}
        for row, col, value in zip(B.r, B.c, B.v):
            if row not in other_dict:
                other_dict[row] = []
            other_dict[row].append((col,value))

        #initialize final matrix parts
        new_vals: np.array = []
        new_cols: np.array = []
        new_rows: np.array = []
        
        #do the matrix multiplication
        acc: dict = {}
        for i in range (A.n):
            if A.c[i] in other_dict:
                for col_b, val_b in other_dict[A.c[i]]:
                    key = (A.r[i],col_b)
                    if key not in acc:
                        acc[key] = 0.0
                    acc[key] += A.v[i] * val_b
        
        #rebuild new SparseMatrix
        for (row,col), val in acc.items():
            if val != 0.0:
                new_rows.append(row)
                new_cols.append(col)
                new_vals.append(val)
                
        return SparseMatrix(new_vals,new_cols,new_rows,A.rs)



    def pow_sparse(self, k:int) -> "SparseMatrix":
        """
        Inputs:
        self: Sparse matrix to be raised to some power
        k: The power to raise the matrix to
        __________________________
        Output:
        Sparse matrix result of raising self to k
        """
        #make identity matrix
        A: "SparseMatrix" = SparseMatrix(np.ones(self.rs),
                                         np.array(range(0,self.rs)),
                                         np.array(range(0,self.rs)),self.rs)
        An: "SparseMatrix" = self
        
        #raise self to the kth power
        while (k > 0):
            if k%2 != 0:
                A = SparseMatrix.sparse_times_sparse(A,An)
            An= SparseMatrix.sparse_times_sparse(An,An)
            k //= 2
        return A
    
    def sparse_times_dense(self, other: np.array) -> np.array:
        """
        Inputs:
        self: sparse matrix to be multiplied
        other: dense matrix to multiply self by
        __________________________
        Output:
        Dense matrix result of the multiplication
        """
        #create output array
        n: int = other.shape[1]
        final: np.array = np.zeros((self.rs,n))
            
        #do the matrix multiplication
        for i,value in enumerate(self.v):
            for col in range(n):
                final[self.r[i], col] += value * other[self.c[i], col]
        
        return final
    
    def cholesky_decomp(A: "SparseMatrix") -> "SparseMatrix":
        """
        Input:
        A: Symmetrical SparseMatrix to be solved
        __________________________
        Output:
        L the lower triangular matrix of A
        """
        #initialize final matrix parts
        new_vals: np.array = []
        new_cols: np.array = []
        new_rows: np.array = []
        
        #make A and L into a dictionary for easy look up
        A_dict = {}
        for val, row, col in zip(A.v, A.r, A.c):
            A_dict[(row, col)] = val
        L = {}
        
        #do cholesky
        for i in range(A.rs):
            for j in range(i+1):
                summ: float =0
                if i==j:
                    for k in range(j):
                        if (i,k) in L:
                            summ += L[(i,k)]**2
                    Aii:float = 0
                    if (i,i) in A_dict: Aii = A_dict[(i,i)]
                    L[(i,i)] = np.sqrt(Aii - summ)
                
                else:
                    if (j,j) in L:
                        for k in range(j):
                            if (i,k) in L and (j,k) in L:
                                summ += L[(i,k)]*L[(j,k)]
                        Aij:float = 0
                        if (i,j) in A_dict: Aij = A_dict[(i,j)]
                        L[(i,j)] = (Aij - summ) / L[(j,j)]
    
        #make the sparse Matrix
        for index in L:
            new_vals.append(L[index])
            new_cols.append(index[1])
            new_rows.append(index[0])
            
        return SparseMatrix(new_vals,new_cols,new_rows,A.rs)
    
    def sparse_transpose(self) -> "SparseMatrix":
        """
        Input:
        self: self matrix to find transpose of
        __________________________
        Output:
        transpose of self
        """
        #this was my favorite method to code
        return SparseMatrix(self.v,self.r,self.c,self.rs)

    def slice_sparse(self, rows: np.array, cols: np.array) -> "SparseMatrix":
        """
        Input:
        self: SparseMatrix to be sliced up
        rows: set of row indices to keep
        cols: set of column indices to keep
        ______________________
        Output:
        SparseMatrix all sliced up
        """
        #make sets for easy access
        row_set: set= set(rows)
        col_set: set= set(cols)
        
        #initialize final matrix parts
        new_vals: np.array = []
        new_cols: np.array = []
        new_rows: np.array = []
            
        #map indices into new positions
        row_map = {old: new for new, old in enumerate(rows)}
        col_map = {old: new for new, old in enumerate(cols)}
        
        for i,val in enumerate(self.v):
            if self.r[i] in row_set and self.c[i] in col_set:
                new_vals.append(val)
                new_cols.append(col_map[self.c[i]])
                new_rows.append(row_map[self.r[i]])
        
        #mapping may switch the order up, so lets sort by rows, then columns within those rows
        entries = [(new_rows[i], new_cols[i], new_vals[i])
                   for i in range(len(new_vals))]

        #sort lexicographically: first by row, then by column
        entries.sort(key=lambda t: (t[0], t[1]))

        #unpack
        rows_sorted, cols_sorted, vals_sorted = zip(*entries)
        new_rows = np.array(rows_sorted)
        new_cols = np.array(cols_sorted)
        new_vals = np.array(vals_sorted)
        return SparseMatrix(new_vals, new_cols, new_rows, len(rows))
    
    def solve_sparse(A: "SparseMatrix", B: np.array) -> np.array:
        """
        Input:
        A: SparseMatrix of sliced up Laplacian
        B: -1(A(boundary)bc)
        __________________________
        Output:
        x so that Ax=b

        Runs in linear time
        """
        # LU decomposition
        superLU = A.cheaters_LU()
        L: "SparseMatrix" = superLU[0]
        U: "SparseMatrix" = superLU[1]
        perm_r: np.array = superLU[2]
        perm_c: np.array = superLU[3]

        # Forward substitution
        L_dict = {}
        row_access = defaultdict(list)
        for val, row, col in zip(L.v, L.r, L.c):
            L_dict[(row, col)] = val
            row_access[row].append((val, col))

        y = np.zeros(L.rs)
        
        #apply permutation matrix
        inv_pr = np.empty_like(perm_r)
        inv_pr[perm_r] = np.arange(len(perm_r))
        B = B[inv_pr]
        
        for i in range(L.rs):
            summ = 0.0
            for value, j in row_access[i]:
                if j < i:
                    summ += value * y[j]
            y[i] = (B[i] - summ) / L_dict[(i,i)]

        # Backward substitution
        U_dict = {}
        row_access = defaultdict(list)
        for val, row, col in zip(U.v, U.r, U.c):
            U_dict[(row, col)] = val
            row_access[row].append((val, col))

        x = np.zeros(U.rs)

        for i in range(U.rs - 1, -1, -1):
            summ = 0.0
            for value, j in row_access[i]:
                if j > i:
                    summ += value * x[j]
            x[i] = (y[i] - summ) / U_dict[(i,i)]
        
        #apply permutation matrix and return
        return x[perm_c]

    def cheaters_LU(self) -> tuple["SparseMatrix",np.array]:
        """
        Input:
        self: Symmetrical SparseMatrix to decomposed
        __________________________
        Output:
        L the lower triangular matrix of A
        """
        import scipy.sparse as sp
        from scipy.sparse.linalg import splu
        
        # convert to csc
        A = sp.csc_matrix((self.v, (self.r, self.c)), shape=(self.rs, self.rs))
        
        # do lu decomposition
        lu = splu(A)
        
        # convert back to my COO SparseMatrix format
        L = lu.L.tocoo()
        U = lu.U.tocoo()
        return (SparseMatrix(L.data, L.col, L.row, L.shape[0]),SparseMatrix(U.data, U.col, U.row, U.shape[0]),
               lu.perm_r,lu.perm_c)
        
        

In [ ]:
def angleBetween(v1: np.array, v2: np.array) -> float:
    """
    Inputs:
    v1,v2: Two vectors 
    ____________________________
    Output:
    Angle between the two vectors
    """
    cosTheta: float = np.dot(v1,v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
    return np.arccos(cosTheta)

def buildLaplacian(v: np.array, f: np.array) -> SparseMatrix:
    """
    Inputs:
    v: Array of vertices
    f: Array of faces
    _____________________________
    Output:
    l: sparse Laplacian matrix
    """
    n = v.shape[0]
    weights = defaultdict(float)
    
    for (i,j,k) in f:
        #angles at i, j, k
        alpha: float = angleBetween(v[j]-v[i], v[k]-v[i]) #angle at i
        beta: float  = angleBetween(v[i] - v[j], v[k] - v[j])  # angle at j
        gamma: float = angleBetween(v[j] - v[k], v[i] - v[k])  # angle at k
        
        #compute their cotangents
        cot_alpha, cot_beta, cot_gamma = 1/np.tan(alpha), 1/np.tan(beta), 1/np.tan(gamma)
        
        #the vertices that make up each edge and the weights opposite of them
        edges: np.array = [(j,k,cot_alpha),(k,i,cot_beta),(i,j,cot_gamma)]
        
        for edge in edges:
            a, b, cot = edge
            weight = .5*cot
            
            #off-diagonals, symmetric so accumulate on both sides of diagonal
            weights[(a,b)] -= weight
            weights[(b,a)] -= weight
    
    #diagonals are sums of each row
    for (i,j), w in list(weights.items()):
        if i != j:
            weights[(i,i)] += -w
        
    #unpack everything into COO sparse format
    rows, cols, vals = zip(*((i, j, w) for (i, j), w in weights.items()))
    #sort everything in row then col order
    order = np.lexsort((cols, rows))
    
    values = np.array(vals)[order]
    rows = np.array(rows)[order]
    columns = np.array(cols)[order]
        
            
    return SparseMatrix(values,columns,rows,n)